# What were the longest and shortest journal article titles in NHB at 2022?

## Choosing the appropriate data source

### Things to consider

* What type of data are we interested in for the given meta-research study? For example, we could be interested in the publication date, author names, number of citations, and the full text of the paper, etc.
  * Scientific databases vary in the types of data they collect and make available to users.
  * We may have to use multiple databases to get all the desired variables.
  * Check the documentation of the databases for further details.
  * Not all databases have the same data quality!
* Is the database of your choice proprietary?
  * Some databases are free to use, while others require a subscription. Ask your institution's librarian to find out whether you have access to the database.
  * Some databases are free to use online (e.g. dimensions.ai) but the API access is restricted. Not the best option if one wants reproducible workflows.
* Is there a readily available API and preferably a wrapper library in your programming language of choice to access the database easily?
  * Using an API to access the data in the database is a secure, structured, scalable, and reproducible method compared to other ways (downloading unstructured data manually for example).
  * Sometimes it is necessary to create custom solutions to solve our problems but in most cases, there is no need to reinvent the wheel.
    * Most programming languages that data scientists use (e.g. Python or R) most likely already have wrapper libraries to use the API with ease. 

### Example

* We use the [PubMed](https://pubmed.ncbi.nlm.nih.gov/) and the [PubMed Central](https://www.ncbi.nlm.nih.gov/pmc/) databases in our example.

## Setting up the project

### Things to consider

* Good practices for collaboration with other and future self.
* What programming language to use?
  * Mostly R or Python.
* Package versioning and virtual environments
  * Package versions can change over time hindering reproducibility of the code.
  * Other packages installed on your machine can influence the results you get when running your code.
  * This could be a source of error and it is hard to track.   
* What IDE?
  * Provides useful tools and integrated solutions to simplify workflow.
  * Jupyter notebook is a popular choice for Python
    * Markdown, Python codechunks and inline code.
    * Writing human-readable output documents with reproducible code.
* README.md
  * First document to check for a given project.
  * Describes the project and folder structure on a high-level.
  * Describes the steps to reproduce the project.
* License
* Version control
  * Tracking changes without clutter.
  * Helps in identifying mistakes and correcting them.
  * Helps with collaboration on code.
  * Using it locally or publishing it to an online repository (e.g. Github or Gitlab).
  * Not all files have to be tracked (use `.gitignore` to exclude files from tracking).
* Project folder structure
  * What goes where within the project?
  * Is there any field-wise or programming language specific standard that we can follow?
    * For example, R packages have a well-documented standard structure. There are other packages that we can take advantage of, which build on that structure. Easier to understand by others without prior knowledge of our project.

### Example

* We use Python in our example. [See this tutorial for help on how to install it.](https://realpython.com/installing-python/).
* Check if Python is successfully installed by using `python -version` or `python3 -V` in your terminal.

* Create a folder for the project `mkdir [folder_name]`
* Open the folder `cd [folder_name]`

* Install `pipenv` to create a virtual working environment by typing `pip install pipenv` in your terminal
  * From now on, all packages should by installed by using the virtual environment, i.e. `pipenv install [package_name]`

* Install `jupyter notebook` by using `pipenv install jupyter`
* Run the `jupyter notebook` by `pipenv run jupyter notebook`

* Initialize the repository as a git repository `git init`
* Add the `.gitignore` file.
* Stage all the files for tracking `git add .`
* Commit the initial changes `git commit -m "init"`

## Writing the code

### Things to consider

* The first version of the code does not have to be perfect.
* We cannot anticipate everything.
* We should first focus on:
  * Creating broad code structure that focuses on funcionality.
  * Identifying potential sources of errors.
* Errors will happen, so it is imporant to catch the errors and log everything so we can learn from them.
* Break down code to functional sections (e.g. retreiving paper ids, getting their metadata, filtering the metadata, etc.)
* We can update the code iteratively to be more precise (within reasonable limits...).

### Example

#### Setup

Load packages.

In [ ]:
from Bio import Entrez
import pandas as pd
import matplotlib.pyplot as plt

Introducing myself to Entrez according to PubMed API rules.

In [ ]:
Entrez.email = "marton.balazs.kovacs@gmail.com"

#### Retrieving the sample

In [ ]:
# Constructing the query
query = f"nature human behaviour[Journal] AND 2022[Publication Date]"

# Create the search handle
handle = Entrez.esearch(db="pubmed", term=query, retmode="xml")

# Rerieve the results
all_results = Entrez.read(handle)
handle.close()

In [ ]:
all_results.keys()

In [ ]:
# The number of papers published in plos one in 2022
all_results['Count']

In [ ]:
len(all_results['IdList'])

The number of ids is not equal to the number of matches!

According to the API documentation the maximum number of results returned is 10thousand. By setting the `retmax` parameter we can get all the results.

In [ ]:
# Constructing the query
query = f"nature human behaviour[Journal] AND 2022[Publication Date]"

# Create the search handle
handle = Entrez.esearch(db="pubmed", term=query, retmode="xml", retmax=9999)

# Rerieve the results
results = Entrez.read(handle)
handle.close()

# Save all the ids
paper_ids = results['IdList']
len(paper_ids)

#### Get the paper information

If we post the ids first on PubMed the retrieval will be faster!

In [ ]:
handle = Entrez.epost(db="pubmed", id=",".join(paper_ids))
post = Entrez.read(handle)
handle.close()
post

In [ ]:
# Setting variables for iteration
batch_size = 10
results = []
total_count = len(paper_ids)

for start in range(0, total_count, batch_size):
    # Set end for iteration
    end = min(total_count, start + batch_size)
    # Print progress
    print(f"Going to download record {start + 1} to {end}")
    # Fetch pubmed article data
    handle = Entrez.efetch(
        db="pubmed",
        retmode="XML",
        retstart=start,
        retmax=batch_size,
        webenv=post["WebEnv"],
        query_key=post["QueryKey"]
    )
    
    # Read returned XML as python object
    data = Entrez.read(handle)
    fetch_handle.close()
    results.append(data)

#### Understanding the structure of the returned data

In [ ]:
len(results)

The `results` array contains 27 batches of PubMed articles. Each batch contains on object of two arrays: 'PubmedBookArticle' and 'PubmedArticle'.

Let's look at the structure of the first batch.

In [ ]:
results[0]

How many articles were in the first batch?

In [ ]:
len(results[0]['PubmedArticle'])

Let's see the first article in the first batch.

In [ ]:
results[0]['PubmedArticle'][0]

In [ ]:
results[0]['PubmedArticle'][0]['MedlineCitation']['Article'].keys()

#### Extracting the measures needed

We can extract the results into a dataframe.

In [ ]:
df = pd.DataFrame(columns=["pubmedid", "title", "message"])

In [ ]:
for data in results: 
    # Get the PubmedArticle array
    # We do not care about books now
    pubmedarticles = data['PubmedArticle']
    
    # Process pubmedarticles if there is any
    if len(pubmedarticles)>0:
        for article in pubmedarticles:
            # Get pubmedid of the result
            pubmed_id = str(article['MedlineCitation']['PMID'])
            # Check if there is a title
            if 'ArticleTitle' not in article['MedlineCitation']['Article']:
                df = pd.concat([df, pd.DataFrame([{"pubmedid": pubmed_id, "title": pd.NA, "message": 'No title found.'}])])
            else :
                df = pd.concat([df, pd.DataFrame([{"pubmedid": pubmed_id, "title": article['MedlineCitation']['Article']['ArticleTitle'], "message": pd.NA}])])
                

Number of extracted articles:

In [ ]:
len(df)

#### Data wrangling

Number of titles extracted:

In [ ]:
len(df['title'].notna())

In [ ]:
df.head()

Let's see the errors if there are any.

In [ ]:
df[df['message'].notna()]

In [ ]:
df['message'].isna().sum()

Add a new column that contains the number of characters in the title column.

In [ ]:
df['n_char'] = df['title'].str.len()
# df['message'].apply(lambda x: len(str(x)) if pd.notna(x) else pd.NA)

#### Results

In [ ]:
f"The longest title was {df['n_char'].max()} characters long while the shortest title was {df['n_char'].min()} characters long."

In [ ]:
plt.hist(df['n_char'].dropna(), bins=20, edgecolor='black')

# Add labels and title
plt.xlabel('Number of characters in the title of the paper')
plt.ylabel('Frequency')

# Show the plot
plt.show()